<!--
authors: Matthew Wilson, Daniele Reda
created: 2020/01/14
last_updated: 2023/02/08
-->


## CPSC 533V: Assignment 3 - Tabular Q Learning and DQN (Due Thu Feb 22)

---

#  Part 1 [54 pts] Tabular Q-Learning 

Tabular Q-learning is an RL algorithm for problems with discrete states and discrete actions. The algorithm is described in the class notes, which borrows the summary description from [Section 6.5](http://incompleteideas.net/book/RLbook2018.pdf#page=153) of Richard Sutton's RL book. In the tabular approach, the Q-value is represented as a lookup table. As discussed in class, Q-learning can further be extended to continuous states and discrete actions, leading to the [Atari DQN](https://arxiv.org/abs/1312.5602) / Deep Q-learning algorithm.  However, it is important and informative to first fully understand tabular Q-learning.

Informally, Q-learning works as follows: The goal is to learn the optimal Q-function: 
`Q(s,a)`, which is the *value* of being at state `s` and taking action `a`.  Q tells you how well you expect to do, on average, from here on out, given that you act optimally.  Once the Q function is learned, choosing an optimal action is as simple as looping over all possible actions and choosing the one with the highest Q (optimal action $a^* = \text{argmax}_a Q(s,a)$).  To learn Q, we initialize it arbitrarily and then iteratively refine it using the Bellman backup equation for Q functions, namely: 
$Q(s,a) \leftarrow Q(s,a) + \alpha [r + \gamma \text{max}_a Q(s', a) - Q(s,a)]$.
Here, $r$ is the reward associated with with the transition from state s to s', and $\alpha$ is a learning rate.

In the first part of assignment you will implement tabular Q-learning and apply it to CartPole -- an environment with a **continuous** state space.  To apply the tabular method, you will need to discretize the CartPole state space by dividing the state-space into bins.


**Goals:**
- to become familiar with python/numpy, as well as using an OpenAI Gym environment
- to understand tabular Q-learning, by implementing tabular Q-Learning for 
  a discretized version of a continuous-state environment, and experimenting with the implementation
- (optional) to develop further intuition regarding possible variations of the algorithm

## Introduction
Deep reinforcement learning has generated impressive results for board games ([Go][go], [Chess/Shogi][chess]), video games ([Atari][atari], [DOTA2][dota], [StarCraft II][scii]), [and][baoding] [robotic][rubix] [control][anymal] ([of][cassie] [course][mimic] ;)).  RL is beginning to work for an increasing range of tasks and capabilities.  At the same time, there are many [gaping holes][irpan] and [difficulties][amid] in applying these methods. Understanding deep RL is important if you wish to have a good grasp of the modern landscape of control methods.

These next several assignments are designed to get you started with deep reinforcement learning, to give you a more close and personal understanding of the methods, and to provide you with a good starting point from which you can branch out into topics of interest. You will implement basic versions of some of the important fundamental algorithms in this space, including Q-learning and policy gradient/search methods.

We will only have time to cover a subset of methods and ideas in this space.
If you want to dig deeper, we suggest following the links given on the course webpage.  Additionally we draw special attention to the [Sutton book](http://incompleteideas.net/book/RLbook2018.pdf) for RL fundamentals and in depth coverage, and OpenAI's [Spinning Up resources](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html) for a concise intro to RL and deep RL concepts, as well as good comparisons and implementations of modern deep RL algorithms.


[atari]: https://arxiv.org/abs/1312.5602
[go]: https://deepmind.com/research/case-studies/alphago-the-story-so-far
[chess]:https://deepmind.com/blog/article/alphazero-shedding-new-light-grand-games-chess-shogi-and-go 
[dota]: https://openai.com/blog/openai-five/
[scii]: https://deepmind.com/blog/article/AlphaStar-Grandmaster-level-in-StarCraft-II-using-multi-agent-reinforcement-learning
[baoding]: https://bair.berkeley.edu/blog/2019/09/30/deep-dynamics/
[rubix]: https://openai.com/blog/solving-rubiks-cube/
[cassie]: https://www.cs.ubc.ca/~van/papers/2019-CORL-cassie/index.html
[mimic]: https://www.cs.ubc.ca/~van/papers/2018-TOG-deepMimic/index.html
[anymal]: https://arxiv.org/abs/1901.08652


[irpan]: https://www.alexirpan.com/2018/02/14/rl-hard.html
[amid]: http://amid.fish/reproducing-deep-rl



In [1]:
# # uncomment if necesary
# !pip install numpy
# !pip install gym
# # OR:
# !pip install gymnasium
import time
import itertools
import numpy as np
# import gym
import gymnasium as gym
import tqdm

---

## [12 pts] Explore the CartPole environment 

Your first task is to familiarize yourself with the OpenAI gym interface and the [CartPole environment](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py)
by writing a simple hand-coded policy to try to solve it.  
To begin understanding OpenAI Gym environments, [read this first](https://gymnasium.farama.org/api/env/).) 
The gym interface is very popular and you will see many algorithm implementations and 
custom environments that support it.  You may even want to use the API in your course projects, 
to define a custom environment for a task you want to solve.

Note that there were several breaking changes introduced in the past few years to the gym API. Some reference algorithm implementations online might be using the old version:
- `obs = env.reset()` ->  `obs, info = env.reset()`
- `obs, reward, done, info = env.step(action)` to `obs, reward, terminated, truncated, info = env.step(action)`
- `env.render()` no longer accepts the `render_mode` parameter (e.g. human mode where the environment is rendered in a pop-up window, or rgb_array which allows headless conversion to images or videos)


Below is some example code that runs a simple random policy.  You are to:
- **run the code to see what it does**
- **write code that chooses an action based on the observation**.  You will need to learn about the gym API and to read the CartPole documentation to figure out what the `action` and `obs` vectors mean for this environment. 
Your hand-coded policy can be arbitrary, and it should ideally do better than the random policy.  There is no single correct answer. The goal is to become familiar with `env`s.
- **write code to print out the total reward gained by your policy in a single episode run**
- **answer the short-response questions below** (see the TODOs for all of this)

In [2]:
env = gym.make('CartPole-v1', render_mode="human")  # you can also try LunarLander-v2, but make sure to change it back
print('observation space:', env.observation_space)
print('action space:', env.action_space)

# To find out what the observations mean, read the CartPole documentation.
# Uncomment the lines below, or visit the source file: 
# https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py

#cartpole = env.unwrapped
#cartpole?

observation space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
action space: Discrete(2)


In [3]:
# 1.1 [10pts]

# runs a single episode and render it.  try running this before editing anything
obs, info = env.reset()  # get first obs/state
rewards = 0
while True:
    # TODO: replace this `action` with something that depends on `obs` 
    if obs[2] > 0:
        action = 1
    else:
        action = 0
    # action = env.action_space.sample()  # random action
    
    obs, reward, terminated, truncated, info = env.step(action)
    rewards += reward
    env.render()
    time.sleep(0.1)  # so it doesn't render too quickly
    if terminated | truncated: break
env.close()

# TODO: print out your total sum of rewards here
print(rewards)

51.0


To answer the questions below, look at the full [source code here](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py) if you haven't already.

**1.2. [2pts] Briefly describe your policy.  What observation information does it use?  What score did you achieve (rough maximum and average)?  And how does it compare to the random policy?**

I used the pole angle. If pole angle > 0, means pole leaning right, so push right to let it maintain balance. Vice versa. Maximum reward is 53, average 44. It's higher than the random policy (reward 9).

---

##  [12 pts] Discretize the env

Next, we need to discretize CartPole's continuous state space to work for tabular Q-learning.  While this is in part  a contrived usage of tabular methods, given the existence of other approaches that are designed to cope with continuous state-spaces, it is also interesting to consider whether tabular methods can be adapted more directly via discretization of the state into bins. Furthermore, tabular methods are simple, interpretabile, and can be proved to converge, and thus they still remain relevant.

Your task is to discretize the state/observation space so that it is compatible with tabular Q-learning.  To do this:
- **implement `obs_normalizer` to pass its test**
- **implement `get_bins` to pass its test**
- **then answer question 2.3**

[map]: https://arxiv.org/abs/1504.04909
[qd]: https://quality-diversity.github.io/

In [4]:
env = gym.make('CartPole-v1')

In [5]:
# 2.1 [5 pts for passing test_normed]
def obs_normalizer(obs):
    """Normalize the observations between 0 and 1
    
    If the observation has extremely large bounds, then clip to a reasonable range before normalizing; 
    (-2,2) should work.  (It is ok if the solution is specific to CartPole)
    
    Args:
        obs (np.ndarray): shape (4,) containing an observation from CartPole using the bound of the env
    Returns:
        normed (np.ndarray): shape (4,) where all elements are roughly uniformly mapped to the range [0, 1]
    
    """
    # HINT: check out env.observation_space.high, env.observation_space.low
    
    # TODO: implement this function
    result = np.zeros_like(obs)
    result[0] = (obs[0] / (2*4.8000002)) + 0.5
    if obs[1] > 2:
        result[1] = 2
    elif obs[1] < -2:
        result[1] = -2
    else:
        result[1] = obs[1]
    result[1] = result[1] / 4 + 0.5
    result[2] = (obs[1] / (2*4.1887903)) + 0.5
    if obs[3] > 2:
        result[3] = 2
    elif obs[3] < -2:
        result[3] = -2
    else:
        result[3] = obs[3]
    result[3] = result[3] / 4 + 0.5
    return result

In [6]:
### TEST 2.1
def test_normed():
    obs, info = env.reset()
    while True:
        obs, _, terminated, truncated, _ =  env.step(env.action_space.sample())
        normed = obs_normalizer(obs) 
        assert np.all(normed >= 0.0) and np.all(normed <= 1.0), '{} are outside of (0,1)'.format(normed)
        if terminated | truncated: break
    env.close()
    print('Passed!')
test_normed()

Passed!


In [7]:
# 2.2 [5 pts for passing test_binned]
def get_bins(normed, num_bins):
    """Map normalized observations (0,1) to bin index values (0,num_bins-1)
    
    Args:
        normed (np.ndarray): shape (4,) output from obs_normalizer
        num_bins (int): how many bins to use
    Returns:
        binned (np.ndarray of type np.int32): shape (4,) where all elements are values in range [0,num_bins-1]
    
    """
    bin_size = 1 / num_bins

    # handle corner case
    normed[normed == 1] -= (0.1 * bin_size)
    return (normed // bin_size).astype(np.int32)

In [8]:
### TEST 2.2
obs, info = env.reset()

def test_binned(num_bins):
    normed = np.array([0.0, 0.2, 0.8, 1.0])
    binned = get_bins(normed, num_bins)
    assert np.all(binned >= 0) and np.all(binned < num_bins), '{} supposed to be between (0, {})'.format(binned, num_bins-1)
    assert binned.dtype == np.int32, "You should also make sure to cast your answer to int using arr.astype(np.int32)" 
    
test_binned(5)
test_binned(10)
test_binned(50)
print('Passed!')

Passed!


**2.3. [2 pts] If your state has 4 values and each is binned into N possible bins, how many bins are needed to represent all unique possible states?**



N<sup>4</sup>

---

## [20 pts] Solve the env 

Using the pseudocode below and the functions you implemented above, implement tabular Q-learning and use it to solve CartPole.

We provide setup code to initialize the Q-table and give examples of interfacing with it. Write the inner and outer loops to train your algorithm.  These training loops will be similar to those deep RL approaches, so get used to writing them!

The algorithm (excerpted from Section 6.5 of [Sutton's book](http://incompleteideas.net/book/RLbook2018.pdf)) is given below:

![Sutton RL](https://i.imgur.com/mdcWVRL.png)

in summary:
- **implement Q-learning using this pseudocode and the helper code**
- **answer the questions below**
- **run the suggested experiments and otherwise experiment with whatever interests you**

In [11]:
env = gym.make('CartPole-v1')
# setup (see last few lines for how to use the Q-table)

# hyper parameters. feel free to change these as desired and experiment with different values
num_bins = 30
alpha = 0.5
gamma = 0.99
log_n = 1000
# epsilon greedy
eps = 0.05  #usage: action = optimal if np.random.rand() > eps else random

obs, info = env.reset()

# Q-table initialized to zeros.  first 4 dims are state, last dim is for action (0,1) for left,right.
Q = np.zeros([num_bins]*len(obs)+[env.action_space.n])

# helper function to convert observation into a binned state so we can index into our Q-table
obs2bin = lambda obs: tuple(get_bins(obs_normalizer(obs), num_bins=num_bins))

s = obs2bin(obs)

print('Shape of Q Table: ', Q.shape) # you can imagine why tabular learning does not scale very well
print('Original obs {} --> binned {}'.format(obs, s))
print('Value of Q Table at that obs/state value', Q[s])

Shape of Q Table:  (30, 30, 30, 30, 2)
Original obs [-0.025777    0.00829503  0.02939578 -0.02648045] --> binned (14, 15, 15, 14)
Value of Q Table at that obs/state value [0. 0.]


In [12]:
# 3.1 [20 pts]

# TODO: implement Q learning, following the pseudo-code above. 
#     - you can follow it almost exactly, but translating things for the gym api and our code used above
#     - make sure to use e-greedy, where e = random about 0.05 percent of the time
#     - make sure to do the S <-- S' step because it can be easy to forget
#     - every log_n steps, you should render your environment and
#       print out the average total episode rewards of the past log_n runs to monitor how your agent trains
#      (your implementation should be able to break at least +150 average reward value, and you can use that 
#       as a breaking condition.  It make take several minutes to run depending on your computer.)

def get_action(state: tuple[int, int, int, int]):
    """Get action based on epsilon greedy"""
    if np.random.random() < eps:
        return env.action_space.sample()
    else:
        return int(np.argmax(Q[state]))
    
def update(
        state: tuple[int, int, int, int],
        action: int,
        reward: float,
        terminated: bool,
        next_state: tuple[int, int, int, int],
    ):
        """Updates the Q-value of an action."""
        future_q_value = (not terminated) * np.max(Q[next_state])
        temporal_difference = (
            reward + gamma * future_q_value - Q[state][action]
        )

        Q[state][action] = (
            Q[state][action] + alpha * temporal_difference
        )


rewards = 0
num_episodes = 0
while True:
    obs, info = env.reset()
    done = False
    num_episodes += 1

    while not done:
        s = obs2bin(obs)
        action = get_action(s)
        next_obs, reward, terminated, truncated, info = env.step(action)
        next_s = obs2bin(next_obs)

        update(s, action, reward, terminated, next_s)
        rewards += reward
        done = terminated or truncated
        obs = next_obs

    if num_episodes % log_n == 0:
        print("The current reward is {}.".format(rewards/num_episodes))
        # env.render()

    if rewards / num_episodes > 150.5:
        break

env.close()

The current reward is 41.445.
The current reward is 57.4995.
The current reward is 68.727.
The current reward is 72.3615.
The current reward is 76.0974.
The current reward is 78.33683333333333.
The current reward is 81.145.
The current reward is 84.83275.
The current reward is 86.13066666666667.
The current reward is 87.7766.
The current reward is 90.81.
The current reward is 91.914.
The current reward is 93.97061538461539.
The current reward is 96.88914285714286.
The current reward is 99.11893333333333.
The current reward is 100.1260625.
The current reward is 101.66235294117647.
The current reward is 103.5195.
The current reward is 104.56005263157894.
The current reward is 105.53015.
The current reward is 106.46419047619048.
The current reward is 107.71972727272727.
The current reward is 108.79086956521739.
The current reward is 109.67795833333334.
The current reward is 110.04208.
The current reward is 111.09026923076924.
The current reward is 111.77588888888889.
The current reward is

## [10 pts] Experiments

Given a working algorithm, you will run a few experiments.  Either make a copy of your code above to modify, or make the modifications in a way that they can be commented out or switched between (with boolean flag if statements).

**4.2. [5 pts] $\epsilon$-greedy.**  How sensitive are the results to the value of $\epsilon$?   First, write down your prediction of what would happen if $\epsilon$ is set to various values, including for example [0, 0.05, 0.25, 0.5].

For $\epsilon=0$, there will be no increasing of reward as there is no exploration and Q is set to all 0s. The policy will choose a fixed action every time, which is suboptimal.

For $\epsilon=0.05$, there will be some exploration. The training will proceed. Reward will increase.

As $\epsilon$ increases, the exploration increases during training. The training speed at first might be slower because of the exploration, but later when it is closer to converge, the policy will provide more accurate actions. However, too high of $\epsilon$ can result in instability of training.

Now run the experiment and observe the impact on the algorithm.  Report the results below.

As expected, no exploration will result in a suboptimal policy. The average reward will stuck at 9.4.

For $\epsilon=0.05$ the rewards accumulate a lot faster at the beginning compared to higher $\epsilon$. This is expected as well since higher $\epsilon$ is doing more exploration than exploitation.

On a 30mins training, 0.05 session reaches avg reward of 150. 0.25 session only reaches 102.

When $\epsilon$ is too high, i.e. 0.25 or 0.5, it's really hard for the policy to converge.

0.01 - 0.1 should be a reasonable range.

**4.3. [5 pts] Design your own experiment.** Design a modification that you think would either increase or reduce performance.  A simple example (which you can use) is initializing the Q-table differently, and thinking about how this might alter performance. Write down your idea, what you think might happen, and why.

My modification is to increase the number of bins. The discretization could impact the result quite a lot. If the number of bin is too low, there might not be enough discretized states to represent the continuous states, i.e. the accuracy of the discretization is low. If the number of bins are too high, the training cost (memory and speed) will increase tremendously due to the exponential increasing of the Q table size. This will also harm since many of the states will result in similar actions so too many states will not increase the performance.

Personally, I think 10 bins didn't perform well on my machine so I decided to try out 20, 30, 40.

Run the experiment and report the results.

It turns out #bins = 30 gives the best performance (speed and avg total reward).

10 somehow stuck at around 135 and increases very slowly after that.

40 trains too slow for the entire time.

---

## A. Extensions (fully optional, will not be graded, if you have time after Part 2)

- plots your learning curve, using e.g., matplotlib 
- visualize the Q-table to see which values are being updated and not
- design a better binning strategy that uses fewer bins for a better-performing policy
- extend this approach to work on different environments (e.g., LunarLander-v2)
- extend this approach to work on environments with continuous actions, by using a fixed set of discrete samples of the action space.  e.g., for Pendulum-v0
- implement a simple deep learning version of this.  we will see next part that DQN uses some tricks to make the neural network training more stable.  Experiment directly with simply replacing the Q-table with a Q-Network and train the Q-Network using gradient descent with `loss = (targets - Q(s,a))**2`, where `targets = stop_grad(R + gamma * maxa(Q(s,a))`).

# Part 2 [60 pts] Behavioral Cloning and Deep Q Learning

---
The second part of assignment will help you transition from tabular approaches to deep neural network approaches. You will implement the [Atari DQN / Deep Q-Learning](https://arxiv.org/abs/1312.5602) algorithm, which arguably kicked off the modern Deep Reinforcement Learning craze.

In this part we will use PyTorch as our deep learning framework.  To familiarize yourself with PyTorch, your first task is to use a behavior cloning (BC) approach to learn a policy.  Behavior cloning is a supervised learning method in which there exists a dataset of expert demonstrations (state-action pairs) and the goal is to learn a policy $\pi$ that mimics this expert.  At any given state, your policy should choose the same action the export would.

Since BC avoids the need to collect data from the policy you are trying to learn, it is relatively simple. 
This makes it a nice stepping stone for implementing DQN. Furthermore, BC is relevant to modern approaches---for example its use as an initialization for systems like [AlphaGo][go] and [AlphaStar][star], which then use RL to further adapte the BC result.  

<!--

I feel like this might be better suited to going lower in the document:

Unfortunately, in many tasks it is impossible to collect good expert demonstrations, making

it's not always possible to have good expert demonstrations for a task in an environemnt and this is where reinforcement learning comes handy. Through the reward signal retrieved by interacting with the environment, the agent learns by itself what is a good policy and can learn to outperform the experts.

-->

Goals:
- Famliarize yourself with PyTorch and its API including models, datasets, dataloaders
- Implement a supervised learning approach (behavioral cloning) to learn a policy.
- Implement the DQN objective and learn a policy through environment interaction.

[go]:  https://deepmind.com/research/case-studies/alphago-the-story-so-far
[star]: https://deepmind.com/blog/article/alphastar-mastering-real-time-strategy-game-starcraft-ii

## Submission information

- Complete by editing and executing the associated Python files.
- Copy and paste the code and the terminal output requested in the predefined cells on this Jupyter notebook.
- When done, upload the completed Jupyter notebook (ipynb file) on canvas.

## Preliminaries

### PyTorch

If you have never used PyTorch before, we recommend you follow this [60 Minutes Blitz][blitz] tutorial from the official website. It should give you enough context to be able to complete the assignment.


**If you have issues, post questions to Piazza**

### Installation

To install all required python packages:

```
python3 -m pip install -r requirements.txt
```

### Debugging


You can include:  `import ipdb; ipdb.set_trace()` in your code and it will drop you to that point in the code, where you can interact with variables and test out expressions.  We recommend this as an effective method to debug the algorithms.


[blitz]: https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

## 1. [36 pts] Behavioral Cloning

Behavioral Cloning is a type of supervised learning in which you are given a dataset of expert demonstrations tuple $(s, a)$ and the goal is to learn a policy function $\hat a = \pi(s)$, such that $\hat a = a$.

The optimization objective is $\min_\theta D(\pi(s), a)$ where $\theta$ are the parameters the policy $\pi$, in our case the weights of a neural network, and where $D$ represents some difference between the actions.

---

Before starting, we suggest reading through the provided files.

For Behavioral Cloning, the important files to understand are: `model.py`, `dataset.py` and `bc.py`.

- The file `model.py` has the skeleton for the model (which you will have to complete in the following questions),

- The file `dataset.py` has the skeleton for the dataset the model is being trained with,

- and, `bc.py` will have all the structure for training the model with the dataset.


### [10 pts] 1.1 Dataset

We provide a pickle file with pre-collected expert demonstrations on CartPole from which to learn the policy $\pi$. The data has been collected from an expert policy on the environment, with the addition of a small amount of gaussian noise to the actions.

The pickle file contains a list of tuples of states and actions in `numpy` in the following way:

```
[(state s, action a), (state s, action a), (state s, action a), ...]
```

In the `dataset.py` file, we provide skeleton code for creating a custom dataset. The provided code shows how to load the file.

Your goal is to overwrite the `__getitem__` function in order to return a dictionary of tensors of the correct type.

Hint: Look in the `bc.py` file to understand how the dataset is used.

Answer the following questions:

**[6 pts]** Insert your code in the placeholder below.

In [ ]:
# PLACEHOLDER TO INSERT YOUR __getitem__ method here
def __getitem__(self, index):
    item = self.data[index]
    return {"state": item[0], "action": item[1]}

In [1]:
from dataset import Dataset

myDataset = Dataset("./CartPole-v1_dataset.pkl")


# def getDimensions(self):
#         states = []
#         actions = []
#         for d in self.data:
#             states.append(d[0])
#             actions.append(d[1])
        
#         states = np.array(states)
#         actions = np.array(actions)
#         return len(states), states.shape, actions.shape, np.max(states, axis=0, keepdims=True), np.min(states, axis=0, keepdims=True), np.max(actions,axis=0,keepdims=True), np.min(actions,axis=0,keepdims=True)


print(myDataset.getDimensions())

(99660, (99660, 4), (99660,), array([[2.39948596, 1.84697975, 0.14641718, 0.47143314]]), array([[-0.72267057, -0.43303689, -0.05007198, -0.38122098]]), array([1], dtype=int64), array([0], dtype=int64))


**[2 pt]** How big is the dataset provided?

99660

**[2 pts]** What is the dimensionality of $s$ and what range does each dimension of $s$ span?  I.e., how much of the state space does the expert data cover? What are the dimensionalities and ranges of the action $a$ in the dataset (how much of the action space does the expert data cover)?

From the previous cell, we can see for the states

- 4 dimension
- each value in the state spans [-0.72267057, -0.43303689, -0.05007198, -0.38122098] to [2.39948596, 1.84697975, 0.14641718, 0.47143314]

for the actions

- 1 dimension
- spans 0 to 1 (both actions)




### [5 pts] 1.2 Environment

Recall the state and action space of CartPole, from the previous assignment.

Considering the full state and action spaces, do you think the provided expert dataset has good coverage?  Why or why not? How might this impact the performance of our cloned policy?

The full state space from the definition is -4.8 to 4.9 for positions, -inf to inf for velocities and -0.418 to 0.418 for angles. However, the termination condition is for those are different. For positions, they are (-2.4, 2.4). For angles, they are (-0.2095,0.2095). In the expert data, the positions are from -0.7 to 2.4, which spans most of the state space (before termination) but not very much the left side to the limit. Similar for the angles. Both actions are coverd. I think the expert policy can handle the situations when the pole are leaning right well but not leaning left. If the expert data lack data for part of the state space, the cloned policy will do even worse because the error accumulates during time.

### [14 pts] 1.3 Model

The file `model.py` provides skeleton code for the model. Your goal is to create the architecture of the network by adding layers that map the input to output.

You will need to update the `__init__` method and the `forward` method.

The `select_action` method has already been written for you.  This should be used when running the policy in the environment, while the `forward` function should be used at training time.

- **[10 pts]** Insert your code in the placeholder below.

In [ ]:
# PLACEHOLDER TO INSERT YOUR MyModel class here

class MyModel(nn.Module):
    def __init__(self, state_size, action_size):
        super().__init__()
        # TODO YOUR CODE HERE FOR INITIALIZING THE MODEL
        # Guidelines for network size: start with 2 hidden layers and maximum 32 neurons per layer
        # feel free to explore different sizes
        self.layers = nn.Sequential(
            nn.Linear(state_size, 256, True),
            nn.ReLU(),
            nn.Linear(256, 256, True),
            nn.ReLU(),
            nn.Linear(256, action_size, True),
            )
        

    def forward(self, x):
        x = self.layers(torch.flatten(x, 1))
        return F.log_softmax(x, dim=1)

    def select_action(self, state):
        self.eval()
        x = self.forward(state)
        self.train()
        return x.max(1)[1].view(1, 1).to(torch.long)

Answer the following questions:

- **[2 pts]** What is the dimension and meaning of the input of the network

Input size is 4, this is the state size.

- **[2 pts]** Similarly, describe the output.

There are 2 hidden layers with 256 neurons each and the final layer just maps the output to 2 actions with a log likelihood softmax.


### [7 pts] 1.4 Training

The file `bc.py` is the entry point for training your behavioral cloning model. The skeleton and the main components are already there.

The missing parts for you to do are:

- Initializing the model
- Choosing a loss function
- Choosing an optimizer
- Playing with hyperparameters to train your model.

- **[5 pts]** Insert your code in the placeholder below.

In [ ]:
# PLACEHOLDER FOR YOUR CODE HER
# HOW DID YOU INITIALIZE YOUR MODEL, OPTIMIZER AND LOSS FUNCTIONS? PASTE HERE YOUR FINAL CODE
# NOTE: YOU CAN KEEP THE FOLLOWING LINES COMMENTED OUT, AS RUNNING THIS CELL WILL PROBABLY RESULT IN ERRORS

model = MyModel(state_size=4, action_size=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = torch.nn.NLLLoss()

You can run your code by doing:

```
python3 bc.py
```

**During all of this assignment, the code in `eval_policy.py` will be your best friend.** At any time, you can test your model by giving as argument the path to the model weights and the environment name using the following command:

```
python3 eval_policy.py --model-path /path/to/model/weights --env ENV_NAME
````

In [ ]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10
[epoch    1/100] [iter       0] [loss 0.69814]
[epoch    1/100] [iter     500] [loss 0.04178]
[epoch    1/100] [iter    1000] [loss 0.02584]
[epoch    1/100] [iter    1500] [loss 0.02598]
[epoch    2/100] [iter    2000] [loss 0.01365]
[epoch    2/100] [iter    2500] [loss 0.00593]
[epoch    2/100] [iter    3000] [loss 0.00375]
[Test on environment] [epoch 2/100] [score 244.20]
[epoch    3/100] [iter    3500] [loss 0.00397]
[epoch    3/100] [iter    4000] [loss 0.00393]
[epoch    3/100] [iter    4500] [loss 0.01878]
[epoch    4/100] [iter    5000] [loss 0.00223]
[epoch    4/100] [iter    5500] [loss 0.01809]
[epoch    4/100] [iter    6000] [loss 0.04941]
[Test on environment] [epoch 4/100] [score 232.30]
[epoch    5/100] [iter    6500] [loss 0.00287]
[epoch    5/100] [iter    7000] [loss 0.00088]
[epoch    5/100] [iter    7500] [loss 0.02178]
[epoch    6/100] [iter    8000] [loss 0.00364]
[epoch    6/100] [iter    8500] [loss 0.00400]
[epoch    6/100] [iter    9000] [loss 0.04437]
[Test on environment] [epoch 6/100] [score 217.50]
[epoch    7/100] [iter    9500] [loss 0.01728]
[epoch    7/100] [iter   10000] [loss 0.00480]
[epoch    7/100] [iter   10500] [loss 0.03154]
[epoch    8/100] [iter   11000] [loss 0.00059]
[epoch    8/100] [iter   11500] [loss 0.02310]
[epoch    8/100] [iter   12000] [loss 0.00040]
[Test on environment] [epoch 8/100] [score 224.00]
[epoch    9/100] [iter   12500] [loss 0.02389]
[epoch    9/100] [iter   13000] [loss 0.00175]
[epoch    9/100] [iter   13500] [loss 0.02723]
[epoch    9/100] [iter   14000] [loss 0.00690]
[epoch   10/100] [iter   14500] [loss 0.05248]
[epoch   10/100] [iter   15000] [loss 0.02820]
[epoch   10/100] [iter   15500] [loss 0.02270]
[Test on environment] [epoch 10/100] [score 275.30]
[epoch   11/100] [iter   16000] [loss 0.01192]
[epoch   11/100] [iter   16500] [loss 0.01329]
[epoch   11/100] [iter   17000] [loss 0.00225]
[epoch   12/100] [iter   17500] [loss 0.00082]
[epoch   12/100] [iter   18000] [loss 0.00377]
[epoch   12/100] [iter   18500] [loss 0.00528]
[Test on environment] [epoch 12/100] [score 263.80]
[epoch   13/100] [iter   19000] [loss 0.00684]
[epoch   13/100] [iter   19500] [loss 0.01661]
[epoch   13/100] [iter   20000] [loss 0.00034]
[epoch   14/100] [iter   20500] [loss 0.00059]
[epoch   14/100] [iter   21000] [loss 0.00110]
[epoch   14/100] [iter   21500] [loss 0.00816]
[Test on environment] [epoch 14/100] [score 241.60]
[epoch   15/100] [iter   22000] [loss 0.00062]
[epoch   15/100] [iter   22500] [loss 0.00228]
[epoch   15/100] [iter   23000] [loss 0.00029]
[epoch   16/100] [iter   23500] [loss 0.01094]
[epoch   16/100] [iter   24000] [loss 0.00026]
[epoch   16/100] [iter   24500] [loss 0.05375]
[Test on environment] [epoch 16/100] [score 239.40]
[epoch   17/100] [iter   25000] [loss 0.00177]
[epoch   17/100] [iter   25500] [loss 0.00086]
[epoch   17/100] [iter   26000] [loss 0.03156]
[epoch   18/100] [iter   26500] [loss 0.01736]
[epoch   18/100] [iter   27000] [loss 0.01559]
[epoch   18/100] [iter   27500] [loss 0.00058]
[epoch   18/100] [iter   28000] [loss 0.00420]
[Test on environment] [epoch 18/100] [score 280.70]
[epoch   19/100] [iter   28500] [loss 0.00018]
[epoch   19/100] [iter   29000] [loss 0.00095]
[epoch   19/100] [iter   29500] [loss 0.00021]
[epoch   20/100] [iter   30000] [loss 0.01053]
[epoch   20/100] [iter   30500] [loss 0.00095]
[epoch   20/100] [iter   31000] [loss 0.00593]
[Test on environment] [epoch 20/100] [score 284.20]
[epoch   21/100] [iter   31500] [loss 0.01220]
[epoch   21/100] [iter   32000] [loss 0.01184]
[epoch   21/100] [iter   32500] [loss 0.00941]
[epoch   22/100] [iter   33000] [loss 0.00003]
[epoch   22/100] [iter   33500] [loss 0.00539]
[epoch   22/100] [iter   34000] [loss 0.00616]
[Test on environment] [epoch 22/100] [score 243.70]
[epoch   23/100] [iter   34500] [loss 0.00131]
[epoch   23/100] [iter   35000] [loss 0.00922]
[epoch   23/100] [iter   35500] [loss 0.00979]
[epoch   24/100] [iter   36000] [loss 0.00263]
[epoch   24/100] [iter   36500] [loss 0.00003]
[epoch   24/100] [iter   37000] [loss 0.00321]
[Test on environment] [epoch 24/100] [score 270.40]
[epoch   25/100] [iter   37500] [loss 0.02302]
[epoch   25/100] [iter   38000] [loss 0.03020]
[epoch   25/100] [iter   38500] [loss 0.00001]
[epoch   26/100] [iter   39000] [loss 0.00005]
[epoch   26/100] [iter   39500] [loss 0.02087]
[epoch   26/100] [iter   40000] [loss 0.00167]
[epoch   26/100] [iter   40500] [loss 0.01376]
[Test on environment] [epoch 26/100] [score 281.00]
[epoch   27/100] [iter   41000] [loss 0.01028]
[epoch   27/100] [iter   41500] [loss 0.04407]
[epoch   27/100] [iter   42000] [loss 0.00744]
[epoch   28/100] [iter   42500] [loss 0.00174]
[epoch   28/100] [iter   43000] [loss 0.00367]
[epoch   28/100] [iter   43500] [loss 0.01550]
[Test on environment] [epoch 28/100] [score 271.00]
[epoch   29/100] [iter   44000] [loss 0.00194]
[epoch   29/100] [iter   44500] [loss 0.00093]
[epoch   29/100] [iter   45000] [loss 0.00076]
[epoch   30/100] [iter   45500] [loss 0.00044]
[epoch   30/100] [iter   46000] [loss 0.00051]
[epoch   30/100] [iter   46500] [loss 0.01051]
[Test on environment] [epoch 30/100] [score 270.70]
[epoch   31/100] [iter   47000] [loss 0.00302]
[epoch   31/100] [iter   47500] [loss 0.00044]
[epoch   31/100] [iter   48000] [loss 0.00029]
[epoch   32/100] [iter   48500] [loss 0.00064]
[epoch   32/100] [iter   49000] [loss 0.00812]
[epoch   32/100] [iter   49500] [loss 0.00023]
[Test on environment] [epoch 32/100] [score 278.10]
[epoch   33/100] [iter   50000] [loss 0.02734]
[epoch   33/100] [iter   50500] [loss 0.00443]
[epoch   33/100] [iter   51000] [loss 0.02590]
[epoch   34/100] [iter   51500] [loss 0.01302]
[epoch   34/100] [iter   52000] [loss 0.00167]
[epoch   34/100] [iter   52500] [loss 0.00004]
[Test on environment] [epoch 34/100] [score 237.90]
[epoch   35/100] [iter   53000] [loss 0.00012]
[epoch   35/100] [iter   53500] [loss 0.00377]
[epoch   35/100] [iter   54000] [loss 0.00012]
[epoch   35/100] [iter   54500] [loss 0.02393]
[epoch   36/100] [iter   55000] [loss 0.00411]
[epoch   36/100] [iter   55500] [loss 0.00299]
[epoch   36/100] [iter   56000] [loss 0.00000]
[Test on environment] [epoch 36/100] [score 233.30]
[epoch   37/100] [iter   56500] [loss 0.00110]
[epoch   37/100] [iter   57000] [loss 0.00020]
[epoch   37/100] [iter   57500] [loss 0.00058]
[epoch   38/100] [iter   58000] [loss 0.00001]
[epoch   38/100] [iter   58500] [loss 0.04285]
[epoch   38/100] [iter   59000] [loss 0.00010]
[Test on environment] [epoch 38/100] [score 247.60]
[epoch   39/100] [iter   59500] [loss 0.00057]
[epoch   39/100] [iter   60000] [loss 0.00195]
[epoch   39/100] [iter   60500] [loss 0.01053]
[epoch   40/100] [iter   61000] [loss 0.00049]
[epoch   40/100] [iter   61500] [loss 0.00113]
[epoch   40/100] [iter   62000] [loss 0.02074]
[Test on environment] [epoch 40/100] [score 233.90]
[epoch   41/100] [iter   62500] [loss 0.04126]
[epoch   41/100] [iter   63000] [loss 0.00806]
[epoch   41/100] [iter   63500] [loss 0.00000]
[epoch   42/100] [iter   64000] [loss 0.00072]
[epoch   42/100] [iter   64500] [loss 0.00162]
[epoch   42/100] [iter   65000] [loss 0.00012]
[Test on environment] [epoch 42/100] [score 264.40]
[epoch   43/100] [iter   65500] [loss 0.00008]
[epoch   43/100] [iter   66000] [loss 0.02011]
[epoch   43/100] [iter   66500] [loss 0.00007]
[epoch   44/100] [iter   67000] [loss 0.00026]
[epoch   44/100] [iter   67500] [loss 0.02528]
[epoch   44/100] [iter   68000] [loss 0.00000]
[epoch   44/100] [iter   68500] [loss 0.00021]
[Test on environment] [epoch 44/100] [score 250.90]
[epoch   45/100] [iter   69000] [loss 0.00012]
[epoch   45/100] [iter   69500] [loss 0.00069]
[epoch   45/100] [iter   70000] [loss 0.00773]
[epoch   46/100] [iter   70500] [loss 0.00007]
[epoch   46/100] [iter   71000] [loss 0.00142]
[epoch   46/100] [iter   71500] [loss 0.00000]
[Test on environment] [epoch 46/100] [score 244.60]
[epoch   47/100] [iter   72000] [loss 0.00170]
[epoch   47/100] [iter   72500] [loss 0.00913]
[epoch   47/100] [iter   73000] [loss 0.00017]
[epoch   48/100] [iter   73500] [loss 0.00771]
[epoch   48/100] [iter   74000] [loss 0.01189]
[epoch   48/100] [iter   74500] [loss 0.01658]
[Test on environment] [epoch 48/100] [score 241.60]
[epoch   49/100] [iter   75000] [loss 0.00004]
[epoch   49/100] [iter   75500] [loss 0.00094]
[epoch   49/100] [iter   76000] [loss 0.00926]
[epoch   50/100] [iter   76500] [loss 0.00091]
[epoch   50/100] [iter   77000] [loss 0.04065]
[epoch   50/100] [iter   77500] [loss 0.03460]
[Test on environment] [epoch 50/100] [score 273.60]
[epoch   51/100] [iter   78000] [loss 0.00004]
[epoch   51/100] [iter   78500] [loss 0.00000]
[epoch   51/100] [iter   79000] [loss 0.03253]
[epoch   52/100] [iter   79500] [loss 0.00001]
[epoch   52/100] [iter   80000] [loss 0.00000]
[epoch   52/100] [iter   80500] [loss 0.00002]
[epoch   52/100] [iter   81000] [loss 0.00001]
[Test on environment] [epoch 52/100] [score 254.50]
[epoch   53/100] [iter   81500] [loss 0.00001]
[epoch   53/100] [iter   82000] [loss 0.00126]
[epoch   53/100] [iter   82500] [loss 0.00116]
[epoch   54/100] [iter   83000] [loss 0.00572]
[epoch   54/100] [iter   83500] [loss 0.00008]
[epoch   54/100] [iter   84000] [loss 0.00428]
[Test on environment] [epoch 54/100] [score 272.20]
[epoch   55/100] [iter   84500] [loss 0.00014]
[epoch   55/100] [iter   85000] [loss 0.00001]
[epoch   55/100] [iter   85500] [loss 0.00062]
[epoch   56/100] [iter   86000] [loss 0.00920]
[epoch   56/100] [iter   86500] [loss 0.00044]
[epoch   56/100] [iter   87000] [loss 0.00239]
[Test on environment] [epoch 56/100] [score 264.50]
[epoch   57/100] [iter   87500] [loss 0.00141]
[epoch   57/100] [iter   88000] [loss 0.00007]
[epoch   57/100] [iter   88500] [loss 0.00124]
[epoch   58/100] [iter   89000] [loss 0.02416]
[epoch   58/100] [iter   89500] [loss 0.00025]
[epoch   58/100] [iter   90000] [loss 0.00050]
[Test on environment] [epoch 58/100] [score 285.60]
[epoch   59/100] [iter   90500] [loss 0.02778]
[epoch   59/100] [iter   91000] [loss 0.00128]
[epoch   59/100] [iter   91500] [loss 0.00183]
[epoch   60/100] [iter   92000] [loss 0.00000]
[epoch   60/100] [iter   92500] [loss 0.00092]
[epoch   60/100] [iter   93000] [loss 0.00003]
[Test on environment] [epoch 60/100] [score 252.60]
[epoch   61/100] [iter   93500] [loss 0.00004]
[epoch   61/100] [iter   94000] [loss 0.01092]
[epoch   61/100] [iter   94500] [loss 0.00002]
[epoch   61/100] [iter   95000] [loss 0.00007]
[epoch   62/100] [iter   95500] [loss 0.00001]
[epoch   62/100] [iter   96000] [loss 0.00918]
[epoch   62/100] [iter   96500] [loss 0.00093]
[Test on environment] [epoch 62/100] [score 278.40]
[epoch   63/100] [iter   97000] [loss 0.00643]
[epoch   63/100] [iter   97500] [loss 0.00033]
[epoch   63/100] [iter   98000] [loss 0.00000]
[epoch   64/100] [iter   98500] [loss 0.00000]
[epoch   64/100] [iter   99000] [loss 0.00172]
[epoch   64/100] [iter   99500] [loss 0.00308]
[Test on environment] [epoch 64/100] [score 253.80]
[epoch   65/100] [iter  100000] [loss 0.00003]
[epoch   65/100] [iter  100500] [loss 0.01033]
[epoch   65/100] [iter  101000] [loss 0.00693]
[epoch   66/100] [iter  101500] [loss 0.00310]
[epoch   66/100] [iter  102000] [loss 0.02156]
[epoch   66/100] [iter  102500] [loss 0.00001]
[Test on environment] [epoch 66/100] [score 240.30]
[epoch   67/100] [iter  103000] [loss 0.00134]
[epoch   67/100] [iter  103500] [loss 0.00178]
[epoch   67/100] [iter  104000] [loss 0.00802]
[epoch   68/100] [iter  104500] [loss 0.00000]
[epoch   68/100] [iter  105000] [loss 0.00715]
[epoch   68/100] [iter  105500] [loss 0.04453]
[Test on environment] [epoch 68/100] [score 299.80]
[epoch   69/100] [iter  106000] [loss 0.00032]
[epoch   69/100] [iter  106500] [loss 0.00006]
[epoch   69/100] [iter  107000] [loss 0.00172]
[epoch   69/100] [iter  107500] [loss 0.00094]
[epoch   70/100] [iter  108000] [loss 0.05337]
[epoch   70/100] [iter  108500] [loss 0.00070]
[epoch   70/100] [iter  109000] [loss 0.00924]
[Test on environment] [epoch 70/100] [score 224.40]
[epoch   71/100] [iter  109500] [loss 0.02305]
[epoch   71/100] [iter  110000] [loss 0.00047]
[epoch   71/100] [iter  110500] [loss 0.03233]
[epoch   72/100] [iter  111000] [loss 0.00001]
[epoch   72/100] [iter  111500] [loss 0.00098]
[epoch   72/100] [iter  112000] [loss 0.00687]
[Test on environment] [epoch 72/100] [score 240.20]
[epoch   73/100] [iter  112500] [loss 0.00428]
[epoch   73/100] [iter  113000] [loss 0.01054]
[epoch   73/100] [iter  113500] [loss 0.04568]
[epoch   74/100] [iter  114000] [loss 0.00000]
[epoch   74/100] [iter  114500] [loss 0.00031]
[epoch   74/100] [iter  115000] [loss 0.03074]
[Test on environment] [epoch 74/100] [score 248.60]
[epoch   75/100] [iter  115500] [loss 0.00001]
[epoch   75/100] [iter  116000] [loss 0.02712]
[epoch   75/100] [iter  116500] [loss 0.00002]
[epoch   76/100] [iter  117000] [loss 0.00000]
[epoch   76/100] [iter  117500] [loss 0.03004]
[epoch   76/100] [iter  118000] [loss 0.00002]
[Test on environment] [epoch 76/100] [score 257.10]
[epoch   77/100] [iter  118500] [loss 0.00019]
[epoch   77/100] [iter  119000] [loss 0.01276]
[epoch   77/100] [iter  119500] [loss 0.00005]
[epoch   78/100] [iter  120000] [loss 0.00001]
[epoch   78/100] [iter  120500] [loss 0.00758]
[epoch   78/100] [iter  121000] [loss 0.00019]
[epoch   78/100] [iter  121500] [loss 0.00233]
[Test on environment] [epoch 78/100] [score 293.80]
[epoch   79/100] [iter  122000] [loss 0.02229]
[epoch   79/100] [iter  122500] [loss 0.00755]
[epoch   79/100] [iter  123000] [loss 0.00759]
[epoch   80/100] [iter  123500] [loss 0.00371]
[epoch   80/100] [iter  124000] [loss 0.00000]
[epoch   80/100] [iter  124500] [loss 0.00020]
[Test on environment] [epoch 80/100] [score 261.50]
[epoch   81/100] [iter  125000] [loss 0.00083]
[epoch   81/100] [iter  125500] [loss 0.00000]
[epoch   81/100] [iter  126000] [loss 0.00660]
[epoch   82/100] [iter  126500] [loss 0.00004]
[epoch   82/100] [iter  127000] [loss 0.01092]
[epoch   82/100] [iter  127500] [loss 0.00170]
[Test on environment] [epoch 82/100] [score 237.30]
[epoch   83/100] [iter  128000] [loss 0.00002]
[epoch   83/100] [iter  128500] [loss 0.00003]
[epoch   83/100] [iter  129000] [loss 0.00002]
[epoch   84/100] [iter  129500] [loss 0.00689]
[epoch   84/100] [iter  130000] [loss 0.00906]
[epoch   84/100] [iter  130500] [loss 0.00001]
[Test on environment] [epoch 84/100] [score 261.70]
[epoch   85/100] [iter  131000] [loss 0.00002]
[epoch   85/100] [iter  131500] [loss 0.00036]
[epoch   85/100] [iter  132000] [loss 0.00583]
[epoch   86/100] [iter  132500] [loss 0.00005]
[epoch   86/100] [iter  133000] [loss 0.00024]
[epoch   86/100] [iter  133500] [loss 0.00013]
[Test on environment] [epoch 86/100] [score 254.30]
[epoch   87/100] [iter  134000] [loss 0.00023]
[epoch   87/100] [iter  134500] [loss 0.02134]
[epoch   87/100] [iter  135000] [loss 0.00137]
[epoch   87/100] [iter  135500] [loss 0.00203]
[epoch   88/100] [iter  136000] [loss 0.00390]
[epoch   88/100] [iter  136500] [loss 0.00000]
[epoch   88/100] [iter  137000] [loss 0.00000]
[Test on environment] [epoch 88/100] [score 262.10]
[epoch   89/100] [iter  137500] [loss 0.00001]
[epoch   89/100] [iter  138000] [loss 0.00085]
[epoch   89/100] [iter  138500] [loss 0.00002]
[epoch   90/100] [iter  139000] [loss 0.00519]
[epoch   90/100] [iter  139500] [loss 0.00361]
[epoch   90/100] [iter  140000] [loss 0.02290]
[Test on environment] [epoch 90/100] [score 262.50]
[epoch   91/100] [iter  140500] [loss 0.00071]
[epoch   91/100] [iter  141000] [loss 0.00083]
[epoch   91/100] [iter  141500] [loss 0.02687]
[epoch   92/100] [iter  142000] [loss 0.00005]
[epoch   92/100] [iter  142500] [loss 0.00004]
[epoch   92/100] [iter  143000] [loss 0.00038]
[Test on environment] [epoch 92/100] [score 288.00]
[epoch   93/100] [iter  143500] [loss 0.00711]
[epoch   93/100] [iter  144000] [loss 0.00116]
[epoch   93/100] [iter  144500] [loss 0.01607]
[epoch   94/100] [iter  145000] [loss 0.00002]
[epoch   94/100] [iter  145500] [loss 0.00000]
[epoch   94/100] [iter  146000] [loss 0.00003]
[Test on environment] [epoch 94/100] [score 264.90]
[epoch   95/100] [iter  146500] [loss 0.00017]
[epoch   95/100] [iter  147000] [loss 0.00000]
[epoch   95/100] [iter  147500] [loss 0.00001]
[epoch   95/100] [iter  148000] [loss 0.00045]
[epoch   96/100] [iter  148500] [loss 0.00000]
[epoch   96/100] [iter  149000] [loss 0.00828]
[epoch   96/100] [iter  149500] [loss 0.03963]
[Test on environment] [epoch 96/100] [score 255.40]
[epoch   97/100] [iter  150000] [loss 0.00000]
[epoch   97/100] [iter  150500] [loss 0.00000]
[epoch   97/100] [iter  151000] [loss 0.00040]
[epoch   98/100] [iter  151500] [loss 0.00932]
[epoch   98/100] [iter  152000] [loss 0.00006]
[epoch   98/100] [iter  152500] [loss 0.00003]
[Test on environment] [epoch 98/100] [score 274.30]
[epoch   99/100] [iter  153000] [loss 0.02656]
[epoch   99/100] [iter  153500] [loss 0.00005]
[epoch   99/100] [iter  154000] [loss 0.02065]
[epoch  100/100] [iter  154500] [loss 0.00000]
[epoch  100/100] [iter  155000] [loss 0.00060]
[epoch  100/100] [iter  155500] [loss 0.00354]
[Test on environment] [epoch 100/100] [score 268.30]
Saving model as behavioral_cloning_CartPole-v1.pt

**[2 pts]** Did you manage to learn a good policy? How consistent is the reward you are getting?

The learned policy is better than the tabular Q learning and faster as well. The average reward is around 250. It has a 40ish variance. Min=210, max= 299 on policy evaluations.

## 2. [24 pts] Deep Q Learning

There are two main issues with the behavior cloning approach.

- First, we are not always lucky enough to have access to a dataset of expert demonstrations.
- Second, replicating an expert policy suffers from compounding error. The policy $\pi$ only sees these "perfect" examples and has no knowledge on how to recover from states not visited by the expert. For this reason, as soon as it is presented with a state that is off the expert trajectory, it will perform poorly and will continue to deviate from a good trajectory without the possibility of recovering from errors.

---
The second task consists in solving the environment from scratch, using RL, and most specifically the DQN algorithm, to learn a policy $\pi$.

For this task, familiarize yourself with the file `dqn.py`. We are going to re-use the file `model.py` for the model you created in the previous task.

Your task is very similar to the one in the previous assignment, to implement the Q-learning algorithm, but in this version, our Q-function is approximated with a neural network.

The algorithm (excerpted from [Atari DQN paper](https://arxiv.org/abs/1312.5602)) is given below:

![DQN algorithm](https://i.imgur.com/Mh4Uxta.png)

### 2.0 [2 pts] Think about your model...



In DQN, we are using the same model as in task 1 for behavioral cloning. In both tasks the model receives as input the state and in both tasks the model outputs something that has the same dimensionality as the number of actions. These two outputs, though, represent very different things. What is each one representing?

**YOUR ANSWER HERE**

### 2.1 [10 pts] Update your Q-function

Complete the `optimize_model` function. This function receives as input a `state`, an `action`, the `next_state`, the `reward` and `done` representing the tuple $(s_t, a_t, s_{t+1}, r_t, done_t)$. Your task is to update your Q-function as shown in the [Atari DQN paper](https://arxiv.org/abs/1312.5602) environment. For now don't be concerned with the experience replay buffer. We'll get to that later.

![Loss function](https://i.imgur.com/tpTsV8m.png)

Insert your code in the placeholder below.

In [ ]:
## PLACEHOLDER TO INSERT YOUR optimize_model function here:

# def optimize_model(state, action, next_state, reward, done):
#     # TODO given a tuple (s_t, a_t, s_{t+1}, r_t, done_t) update your model weights

#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

### 2.2 [5 pts] $\epsilon$-greedy strategy

You will need a strategy to explore your environment. The standard strategy is to use $\epsilon$-greedy. Implement it in the `choose_action` function template.

Insert your code in the placeholder below.

In [ ]:
## PLACEHOLDER TO INSERT YOUR choose_action function here:

# def choose_action(state, test_mode=False):
#     # TODO implement an epsilon-greedy strategy
#     raise NotImplementedError()

### 2.3 [2 pts] Train your model

Try to train a model in this way.

You can run your code by doing:

```
python3 dqn.py
```

How many episodes does it take to learn (ie. reach a good reward)?

**YOUR ANSWER HERE**

In [ ]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10

### 2.4 [5 pts] Add the Experience Replay Buffer

If you read the DQN paper (and as you can see from the algorithm picture above), the authors make use of an experience replay buffer to learn faster. We provide an implementation in the file `replay_buffer.py`. Update the `train_reinforcement_learning` code to push a tuple to the replay buffer and to sample a batch for the `optimize_model` function.

In [ ]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10

How does the replay buffer improve performance?

**YOUR ANSWER HERE**

## 3. Extra (fully optional)

Ideas to experiment with:

- Is $\epsilon$-greedy strategy the best strategy available? Experiment with other strategies.
- Make use of the model you have trained in the behavioral cloning part and fine-tune it with RL. How does that affect performance?
- You are perhaps bored with `CartPole-v1` by now. Another environment we suggest trying is `LunarLander-v2`. It will be harder to learn but with experimentation, you will find the correct optimizations for success. Piazza is also your friend :)
- What about learning from images? This requires more work because you have to extract the image from the environment. How much more challenging might you expect the learning to be in this case?
- An improvement over DQN is DoubleDQN. Experiment with this to see how much of an impact it makes.



In [ ]:
# YOU CAN USE THIS CODEBLOCK AND ADD ANY BLOCK BELOW AS YOU NEED
# TO SHOW US THE IDEAS AND EXTRA EXPERIMENTS YOU RUN.
# HAVE FUN!